# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [10]:
# import libraries
import re
import numpy as np
import pandas as pd
import pickle
from time import time

# import nltk libraries
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# import sklearn libraries
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV 
from sklearn.multioutput import MultiOutputClassifier


from sqlalchemy import create_engine

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Peter\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Peter\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Peter\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('disaster_table', con=engine)

X = df['message'] # reads message column
y = df.iloc[:,4:] # reads all rows, all column starting at [4] ->

In [12]:
# Display message column
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [13]:
# Display categories dataframe made from ETL pipeline
y.head()

,related,request,offer,aid related,medical help,medical products,search and rescue,security,military,child alone,...,aid centers,other infrastructure,weather related,floods,storm,fire,earthquake,cold,other weather,direct report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# Displays length of characters in each message
df['message'] = df['message'].apply(lambda x: len(x))
df['message']

0         66
1         39
2         31
3        100
4         63
        ... 
26210    148
26211    136
26212    141
26213    191
26214    209
Name: message, Length: 26215, dtype: int64

In [15]:
# Index of column names
y.columns

Index(['related', 'request', 'offer', 'aid related', 'medical help',
       'medical products', 'search and rescue', 'security', 'military',
       'child alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing people', 'refugees', 'death', 'other aid',
       'infrastructure related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid centers', 'other infrastructure',
       'weather related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other weather', 'direct report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [16]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

# function to detect url, normalize, tokenize, remove stopwords, lemmatize words 
def tokenize(text, lemmatizer=WordNetLemmatizer()):
    # Detect urls with url_regex
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    # Normalizes and tokenizes (removes punctuations and converts to lowercase)
    tokens = nltk.word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower()))

        # Remove stopwords
    tokens = [t for t in tokens if t not in stopwords.words('english')]

    # lemmatize and remove stop words
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        if clean_tok not in stopwords.words("english"):
            clean_tokens.append(clean_tok)
    pass
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [18]:
# Pipeline with RandomForestClf
pipeline = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier())),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [176]:
# Splits dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [177]:
# Fits pipeline on train set 
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x000001AB7F5FA700>)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced')))])

In [178]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   TfidfVectorizer(tokenizer=<function tokenize at 0x000001AB7F5FA700>)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced')))],
 'verbose': False,
 'vect': TfidfVectorizer(tokenizer=<function tokenize at 0x000001AB7F5FA700>),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced')),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.float64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__norm': 'l2',
 'vect__preprocessor': None,
 'vect__smooth_idf': True,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__sublinear_tf': False,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text, lemmatizer=<WordNetLemmatizer>)>,
 'vect__us

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [179]:
# Predict pipeline X_test (long process)
y_pred = pipeline.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, target_names=y.columns))

### 6. Improve your model
Use grid search to find better parameters. 

In [186]:
# Pipeline 2 with RFC
pipeline2 = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize)),
    ('clf',MultiOutputClassifier(RandomForestClassifier())),
])

In [ ]:
from sklearn.model_selection import GridSearchCV

# Parameters used for GridsearchCV
parameters2 = {'clf__estimator__n_estimators': [10, 30],}

cv2 = GridSearchCV(estimator=pipeline2, param_grid=parameters2, verbose=3)

In [187]:
cv2.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(tokenizer=<function tokenize at 0x000001AB7F5FA700>)),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced')))]),
             param_grid={'clf__estimator__n_estimators': [20, 50]}, verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [188]:
y_pred2 = cv2.predict(X_test)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] clf__estimator__n_estimators=20 .................................
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] ..... clf__estimator__n_estimators=20, score=0.244, total= 6.0min
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  6.0min remaining:    0.0s
[CV] clf__estimator__n_estimators=20 .................................
[CV] ..... clf__estimator__n_estimators=20, score=0.247, total= 6.2min
[CV] clf__estimator__n_estimators=20 .................................[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 12.2min remaining:    0.0s

[CV] ..... clf__estimator__n_estimators=20, score=0.258, total= 6.3min
[CV] clf__estimator__n_estimators=50 .................................
[CV] ..... clf__estimator__n_estimators=50, score=0.244, total= 7.8min
[CV] clf__estimator__n_estimators=50 .................................
[CV] ..... clf__estimator__n_estimators=50, score=0.243, tota

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(tokenizer=<function tokenize at 0x000001AB7F5FA700>)),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced')))]),
             param_grid={'clf__estimator__n_estimators': [20, 50]}, verbose=3)

In [189]:
print(classification_report(y_test, y_pred2, target_names=y.columns))

related
              precision    recall  f1-score   support

           0       0.68      0.47      0.56      1266
           1       0.84      0.93      0.88      3938
           2       0.29      0.40      0.34        40

    accuracy                           0.81      5244
   macro avg       0.61      0.60      0.59      5244
weighted avg       0.80      0.81      0.80      5244

request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      4349
           1       0.82      0.52      0.64       895

    accuracy                           0.90      5244
   macro avg       0.86      0.75      0.79      5244
weighted avg       0.89      0.90      0.89      5244

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0.50      524

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [25]:
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier

# Pipeline 3 with AdaboostClf
pipeline3 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [21]:
# tuning parameters
                
parameters3 = {'tfidf__use_idf': (True, False),
                'clf__estimator__n_estimators': [10, 40], 
               'clf__estimator__min_samples_split': [2, 4],
               'clf__estimator__learning_rate': [1,2]}


cv3 = GridSearchCV(estimator=pipeline3, param_grid=parameters3, verbose=3)

NameError: name 'pipeline3' is not defined

In [201]:
cv3

In [202]:
cv3.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001AB7F5FA700>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier(base_estimator=DecisionTreeClassifier(class_weight='balanced',
                                                                                                                                 max_depth=1))))]),
             param_grid={'clf__estimator__learning_rate': [0.1, 0.3],
                         'clf__estimator__n_estimators': [200, 400]},
             verbose=3)

In [199]:
y_pred3 = cv3.predict(X_test)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=200 
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


ValueError: multiclass-multioutput is not supported

In [ ]:
print(classification_report(y_test, y_pred3, target_names=y.columns))

### 9. Export your model as a pickle file

In [ ]:
with open('classifier.pkl', 'wb') as file:
    pickle.dump(cv2, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.